In [1]:
# libraries import

import ipywidgets as widgets
from ipyfilechooser import FileChooser
import os
import whitebox
from IPython.display import Markdown, display
import geopandas as gpd
from ipyleaflet import Map, basemaps, LayersControl, FullScreenControl, GeoData #smazat?
from rasterstats import zonal_stats
import rasterio
import numpy as np
import copy
from shapely.geometry import LineString
from rasterio.mask import mask
from osgeo import ogr, gdal
import folium
from folium import raster_layers, plugins
from folium.plugins import MeasureControl, MousePosition
from matplotlib import cm
from matplotlib.cm import *
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import branca.colormap as cm

In [2]:
def printmd(string):
    
    display(Markdown(string))

In [3]:
def processing():
    
    global out2
    out2 = widgets.Output(layout={'border': '2px solid black', 'height':'300px'})
    display(out2)
    
    with out2:
        display('will be processed here ..')

In [4]:
def msg():
    
    global out3
    
    out3 = widgets.Output(layout={'border': '2px solid black', 'height':'350px'})
    display(out3)
    
    with out3:
        display('...')

In [5]:
def calculation():
    
    global inter_valley_shp_map
    global basins_out_gdf
    global bas_out
  
    # inputs    
    bas_shp = gpd.read_file(basins) 
    source_basin_shp = gpd.read_file(source_basins) 
    inter_valley_shp = gpd.read_file(inter_valley) 
    source_valleys_shp = gpd.read_file(source_valleys) 
    
    
    # PARAMETERS ---- basins -----------------------------------------------------------------  
    
    # slope of watershed
    slope = 'slope.tif'
    wbt.slope(
        dem, 
        slope, 
        zfactor=None, 
        units="degrees"
    )
    
    slope_path = wdir + slope   
    
    # number of pixels in each watershed
    bas_shp['count0'] = gpd.GeoDataFrame(
        zonal_stats(
            vectors=bas_shp['geometry'], 
            raster=dem, 
            stats='count'
        )
    )['count']    
    
    # Area percentage of watershed with slopes over 35° (Unit: %)
    slope_ras = rasterio.open(slope_path)
    slope_img = slope_ras.read(1)
    slope_crs = slope_ras.crs
    slope_tra = slope_ras.transform
    
    slope35_img = copy.copy(slope_img)
    slope35_img[slope_img<34.9] = np.nan  
        
    slope35path = wdir + 'slope35.tif'
    with rasterio.open(
        slope35path,
        'w',
        driver='GTiff',
        height=slope35_img.shape[0],
        width=slope35_img.shape[1],
        count=1,
        dtype=slope_img.dtype,
        crs=slope_crs,
        transform = slope_tra
    ) as dst:
        dst.write(slope35_img, 1) 
        pxsz35, pysz35 = dst.res
        
    bas_shp['slope35'] = gpd.GeoDataFrame(
        zonal_stats(
            vectors=bas_shp['geometry'], 
            raster=slope35path, 
            stats='count'
        )
    )['count']
    
    # area in m2 of watershed with slopes over 35°
    bas_shp['slope>35'] = bas_shp['slope35'] * pxsz35 * pysz35 
    
    # area percentage of watershed with slopes over 35°
    bas_shp['slope>35_%'] = bas_shp['slope35'] / (bas_shp['count0'] / 100.0) 
    
    with out3:
        display("slopes over 35° - done")
        
    # Shape - Area percentage of watershed at an altitude above 2500 m.a.s.l. (Unit: %)
    dem_ras =  rasterio.open(dem)
    dem_img = dem_ras.read(1)
    dem_crs = dem_ras.crs
    dem_tra = dem_ras.transform
    
    dem2500_img = copy.copy(dem_img)
    dem2500_img[dem_img<2499.9] = np.nan   
    
    dem2500path = wdir + 'dem2500.tif'
    with rasterio.open(
        dem2500path,
        'w',
        driver='GTiff',
        height=dem2500_img.shape[0],
        width=dem2500_img.shape[1],
        count=1,
        dtype=dem_img.dtype,
        crs=dem_crs,
        transform = dem_tra
    ) as dst:
        dst.write(dem2500_img, 1)    
    
    bas_shp['el2500'] = gpd.GeoDataFrame(
        zonal_stats(
            vectors=bas_shp['geometry'], 
            raster=dem2500path, 
            stats='count'
        )
    )['count']
          
    # area percentage of watershed with slopes over 35°
    bas_shp['ele>2500_%'] = bas_shp['el2500'] / (bas_shp['count0'] / 100.0)    
    
    with out3:
        display("altitude above 2500 m.a.s.l. - done")
     
    #relative elevation difference
    #max
    bas_shp['el_max'] = gpd.GeoDataFrame(
        zonal_stats(
            vectors=bas_shp['geometry'], 
            raster=dem, 
            stats='max'
        )
    )['max']
    #min
    bas_shp['el_min'] = gpd.GeoDataFrame(
        zonal_stats(
            vectors=bas_shp['geometry'], 
            raster=dem, 
            stats='min'
        )
    )['min']
    bas_shp['rel_el_dif'] = bas_shp['el_max'] - bas_shp['el_min']  
       
    #profile curvature
    profileCurv_path = wdir + 'prof_curvature.tif'    
    wbt.profile_curvature(
        dem, 
        profileCurv_path   
    )
    
    prof_ras =  rasterio.open(profileCurv_path)
    prof_img = prof_ras.read(1)
    prof_crs = prof_ras.crs
    prof_tra = prof_ras.transform
    
    prof_img = copy.copy(prof_img)
    prof_img[prof_img<-1.0] = np.nan   
    prof_img[prof_img>1.0] = np.nan     
    
    profileCurv_path2 = wdir + 'prof_curvature2.tif'     
    with rasterio.open(
        profileCurv_path2,
        'w',
        driver='GTiff',
        height=prof_img.shape[0],
        width=prof_img.shape[1],
        count=1,
        dtype=prof_img.dtype,
        crs=prof_crs,
        transform = prof_tra
    ) as dst:
        dst.write(prof_img, 1)    
    
    bas_shp['prof_curv'] = gpd.GeoDataFrame(
        zonal_stats(
            vectors=bas_shp['geometry'], 
            raster=profileCurv_path2, 
            stats='mean'
        )
    )['mean']
    
    #plan curvature
    planCurv_path = wdir + 'plan_curvature.tif'    
    wbt.plan_curvature(
        dem, 
        planCurv_path
    )
    
    plan_ras =  rasterio.open(planCurv_path)
    plan_img = plan_ras.read(1)
    plan_crs = plan_ras.crs
    plan_tra = plan_ras.transform
    
    plan_img = copy.copy(plan_img)
    plan_img[plan_img<-1.0] = np.nan   
    plan_img[plan_img>1.0] = np.nan    
    
    planCurv_path2 = wdir + 'plan_curvature2.tif'
    with rasterio.open(
        planCurv_path2,
        'w',
        driver='GTiff',
        height=plan_img.shape[0],
        width=plan_img.shape[1],
        count=1,
        dtype=plan_img.dtype,
        crs=plan_crs,
        transform = plan_tra
    ) as dst:
        dst.write(plan_img, 1)   
    
    bas_shp['plan_curv'] = gpd.GeoDataFrame(
        zonal_stats(
            vectors=bas_shp['geometry'], 
            raster=planCurv_path2, 
            stats='mean'
        )
    )['mean']
    
   
    #ground variability  = profile curv x plan curv
    ground_var_path = wdir + 'gr_variability.tif'
    wbt.multiply(
        profileCurv_path, 
        planCurv_path, 
        ground_var_path
    )
    
    gr_ras =  rasterio.open(ground_var_path)
    gr_img = gr_ras.read(1)
    gr_crs = gr_ras.crs
    gr_tra = gr_ras.transform
    
    gr_img = copy.copy(gr_img)
    gr_img[gr_img<-1.0] = np.nan   
    gr_img[gr_img>1.0] = np.nan    
    
    ground_var_path2 = wdir + 'gr_variability2.tif'
    with rasterio.open(
        ground_var_path2,
        'w',
        driver='GTiff',
        height=gr_img.shape[0],
        width=gr_img.shape[1],
        count=1,
        dtype=gr_img.dtype,
        crs=gr_crs,
        transform = gr_tra
    ) as dst:
        dst.write(gr_img, 1)      
    
    bas_shp['gr_var'] = gpd.GeoDataFrame(
        zonal_stats(
            vectors=bas_shp['geometry'], 
            raster=ground_var_path2, 
            stats='mean'
        )
    )['mean']
    
    #glaciers
    bas_shp['gl_count'] = gpd.GeoDataFrame(
        zonal_stats(
            vectors=bas_shp['geometry'], 
            raster=glac, 
            stats='count'
        )
    )['count']
    
    # area percentage of glacier in basin
    bas_shp['glac_%'] = bas_shp['gl_count'] / (bas_shp['count0'] / 100.0)  
    
    
    
    # PARAMETERS ---- basins -----------------------------------------------------------------
    # ----------------------------------------------------------------------------------------  
     
    
    
    
    # ----------------------------------------------------------------------------------------
    # PARAMETERS ---- source basins -----------------------------------------------------------------  
       
    # number of pixels in each watershed
    source_basin_shp['count0'] = gpd.GeoDataFrame(
        zonal_stats(
            vectors=source_basin_shp['geometry'], 
            raster=dem, 
            stats='count'
        )
    )['count']    
    
    # Area percentage of watershed with slopes over 35° (Unit: %)
    source_basin_shp['slope35'] = gpd.GeoDataFrame(
        zonal_stats(
            vectors=source_basin_shp['geometry'], 
            raster=slope35path, 
            stats='count'
        )
    )['count']
    
    # area in m2 of watershed with slopes over 35°
    source_basin_shp['slope>35'] = source_basin_shp['slope35'] * pxsz35 * pysz35 
    
    # area percentage of watershed with slopes over 35°
    source_basin_shp['slope>35_%'] = source_basin_shp['slope35'] / (source_basin_shp['count0'] / 100.0)    
    
    with out3:
        display("slopes over 35° - done")
        
    # Shape - Area percentage of watershed at an altitude above 2500 m.a.s.l. (Unit: %)
    source_basin_shp['el2500'] = gpd.GeoDataFrame(
        zonal_stats(
            vectors=source_basin_shp['geometry'], 
            raster=dem2500path, 
            stats='count'
        )
    )['count']
    
    
    # area percentage of watershed with slopes over 35°
    source_basin_shp['ele>2500_%'] = source_basin_shp['el2500'] / (source_basin_shp['count0'] / 100.0)    
    
    with out3:
        display("altitude above 2500 m.a.s.l. - done")   
        
    #relative elevation difference
    #max
    source_basin_shp['el_max'] = gpd.GeoDataFrame(
        zonal_stats(
            vectors=source_basin_shp['geometry'], 
            raster=dem, 
            stats='max'
        )
    )['max']
    #min
    source_basin_shp['el_min'] = gpd.GeoDataFrame(
        zonal_stats(
            vectors=source_basin_shp['geometry'], 
            raster=dem, 
            stats='min'
        )
    )['min']
    source_basin_shp['rel_el_dif'] = source_basin_shp['el_max'] - source_basin_shp['el_min']      
    
    #profile curvature  
    source_basin_shp['prof_curv'] = gpd.GeoDataFrame(
        zonal_stats(
            vectors=source_basin_shp['geometry'], 
            raster=profileCurv_path2, 
            stats='mean'
        )
    )['mean']
    #plan curvature  
    source_basin_shp['plan_curv'] = gpd.GeoDataFrame(
        zonal_stats(
            vectors=source_basin_shp['geometry'], 
            raster=planCurv_path2, 
            stats='mean'
        )
    )['mean']
    #ground variability  = profile curv x plan curv    
    source_basin_shp['gr_var'] = gpd.GeoDataFrame(
        zonal_stats(
            vectors=source_basin_shp['geometry'], 
            raster=ground_var_path2, 
            stats='mean'
        )
    )['mean']
    
    #glaciers
    source_basin_shp['gl_count'] = gpd.GeoDataFrame(
        zonal_stats(
            vectors=source_basin_shp['geometry'], 
            raster=glac, 
            stats='count'
        )
    )['count']
    
    # area percentage of glacier in basin
    source_basin_shp['glac_%'] = source_basin_shp['gl_count'] / (source_basin_shp['count0'] / 100.0)  
    
       
    # PARAMETERS ---- source basins -----------------------------------------------------------------
    # ---------------------------------------------------------------------------------------- 
    
      
    
    
    
    # ----------------------------------------------------------------------------------------
    # PARAMETERS ---- talwegs -----------------------------------------------------------------  
    
    
    # INTERSECT VALLEY      
           
    inter_valley_shp['FID'] = np.intc(inter_valley_shp['BASIN'])
    del inter_valley_shp['AVG_SLOPE']    
    
    #pixel size
    pxsz, pysz = slope_ras.res
    px_half = pxsz / 2.0    

    inter_valley_shp = inter_valley_shp.assign(curvature=0.0)
    inter_valley_shp = inter_valley_shp.assign(avg_slope=0.0)
    inter_valley_shp = inter_valley_shp.assign(shortLen=0.0)    
    inter_valley_shp = inter_valley_shp.assign(slope_15=0.0)    
    
    
    #absolutni hodnoty --------------------------------------------
    
    inter_valley_shp = inter_valley_shp.assign(sl15_count=0.0)
    inter_valley_shp = inter_valley_shp.assign(count=0.0)
    inter_valley_shp = inter_valley_shp.assign(bas_area=0.0)
   
    
    #--------------------------------------------------------------
    
    
    for i in np.arange(0, len(inter_valley_shp['FID'])):
        inter_geom = inter_valley_shp.iloc[i]['geometry']    
        arr_line = np.asarray(inter_geom.coords)
        p1 = arr_line[0,:]
        p2 = arr_line[-1,:]
        straight_line = LineString((p1, p2))
        inter_valley_shp.loc[i, 'shortLen'] = straight_line.length
        curve_line = inter_valley_shp['geometry'][i]
        inter_valley_shp.loc[i, 'LENGTH'] = curve_line.length

        # intersect valley average slope        
        buf = inter_valley_shp.geometry[i].buffer(px_half)       
        out_image, out_transform = mask(slope_ras, [buf], crop=True)
        inter_valley_shp.loc[i, 'avg_slope'] = np.mean(out_image[out_image>0.0])

        # Length of valley with slopes over 15°        
        sl_count = len(out_image[out_image>0.0])
        sl15_count = len(out_image[out_image>15.0])        
        inter_valley_shp.loc[i, 'slope_15'] = sl15_count * pxsz
        
        # Length percentage of valley with slopes over 15°       
        inter_valley_shp.loc[i, 'sl_15_%'] = sl15_count / (sl_count / 100.0)
                
        #real values --------------------------------------------
        
        inter_valley_shp.loc[i, 'sl15_count'] = sl15_count
        inter_valley_shp.loc[i, 'count'] = sl_count
        
        #--------------------------------------------------------------   
        
        # basin area
        inter_bas = inter_valley_shp.iloc[i]['BASIN']
        bas_count = bas_shp.loc[bas_shp['VALUE'] == inter_bas, 'count0']      
        inter_valley_shp.loc[i, 'bas_area'] = bas_count.values[0] * pxsz * pxsz
        
    
    with out3:
        display("parameters counted")
                    
    inter_valley_shp['curvature'] = inter_valley_shp['LENGTH'] / inter_valley_shp['shortLen']  
    
    #channel gradient [meter per meter] = ele dif / length    
    inter_valley_shp['channel_gr'] = inter_valley_shp['eleDif'] / inter_valley_shp['LENGTH'] 
    
    #area gradient = ele dif / area    
    inter_valley_shp['area_gr'] = inter_valley_shp['eleDif'] / inter_valley_shp['bas_area'] * 100.0
    
    #shape gradient = length / area    
    inter_valley_shp['shape_gr'] = inter_valley_shp['LENGTH'] / inter_valley_shp['bas_area'] * 100.0
    
    #removal of redudant columns
    del inter_valley_shp['UP_ELEV']; del inter_valley_shp['DN_ELEV']; del inter_valley_shp['LENGTH']; del inter_valley_shp['blind']; del inter_valley_shp['shortLen']; del inter_valley_shp['slope_15']; del inter_valley_shp['sl15_count']; del inter_valley_shp['count']; del inter_valley_shp['bas_area'];
        
    # save talweg
    inter_valley_out =  wdir + 'inter_valley_params.shp'
    inter_valley_shp.to_file(inter_valley_out)  
    
    with out3:
        display("inter_valley_params.shp EXPORTED")   
    
    # PARAMETERS ---- talwegs ----------------------------------------------------------------- 
    # ----------------------------------------------------------------------------------------
     
 
 
    
    # ----------------------------------------------------------------------------------------
    # PARAMETERS ---- source valleys -----------------------------------------------------------------  
    
    # SOURCE VALLEY          
    
    source_valleys_shp['FID'] = np.intc(source_valleys_shp['BASIN'])
    del source_valleys_shp['AVG_SLOPE']    
    
    #pixel size
    pxsz, pysz = slope_ras.res
    px_half = pxsz / 2.0    

    source_valleys_shp = source_valleys_shp.assign(curvature=0.0)
    source_valleys_shp = source_valleys_shp.assign(avg_slope=0.0)
    source_valleys_shp = source_valleys_shp.assign(shortLen=0.0)    
    source_valleys_shp = source_valleys_shp.assign(slope_15=0.0)
    
    
    #real numbers --------------------------------------------
    
    source_valleys_shp = source_valleys_shp.assign(sl15_count=0.0)
    source_valleys_shp = source_valleys_shp.assign(count=0.0)
    
    #--------------------------------------------------------------
    
    source_valleys_shp = source_valleys_shp.assign(bas_area=0.0)
    
    
    for i in np.arange(0, len(source_valleys_shp['FID'])):
        source_geom = source_valleys_shp.iloc[i]['geometry']    
        arr_line = np.asarray(source_geom.coords)
        p1 = arr_line[0,:]
        p2 = arr_line[-1,:]
        straight_line = LineString((p1, p2))
        source_valleys_shp.loc[i, 'shortLen'] = straight_line.length
        curve_line = source_valleys_shp['geometry'][i]
        source_valleys_shp.loc[i, 'LENGTH'] = curve_line.length

        # intersect valley average slope        
        buf = source_valleys_shp.geometry[i].buffer(px_half)       
        out_image, out_transform = mask(slope_ras, [buf], crop=True)
        source_valleys_shp.loc[i, 'avg_slope'] = np.mean(out_image[out_image>0.0])

        # Length of valley with slopes over 15°        
        sl_count = len(out_image[out_image>0.0])
        sl15_count = len(out_image[out_image>15.0])        
        source_valleys_shp.loc[i, 'slope_15'] = sl15_count * pxsz
        
        # Length percentage of valley with slopes over 15°       
        source_valleys_shp.loc[i, 'sl_15_%'] = sl15_count / (sl_count / 100.0)
                
        #areal numbers --------------------------------------------
        
        source_valleys_shp.loc[i, 'sl15_count'] = sl15_count
        source_valleys_shp.loc[i, 'count'] = sl_count
        
        #--------------------------------------------------------------  
        
        # basin area
        source_bas = source_valleys_shp.iloc[i]['BASIN']
        s_bas_count = source_basin_shp.loc[source_basin_shp['VALUE'] == source_bas, 'count0']
        source_valleys_shp.loc[i, 'bas_area'] = s_bas_count.values[0] * pxsz * pxsz
        
       
        
    
    with out3:
        display("parameters counted")
    
    source_valleys_shp['curvature'] = source_valleys_shp['LENGTH'] / source_valleys_shp['shortLen']    
    
    #channel gradient [meter per meter] = ele dif / length 
    source_valleys_shp['channel_gr'] = source_valleys_shp['eleDif'] / source_valleys_shp['LENGTH']
    
    #area gradient = ele dif / area    
    source_valleys_shp['area_gr'] = source_valleys_shp['eleDif'] / source_valleys_shp['bas_area'] * 100.0
    
    #shape gradient = length / area    
    source_valleys_shp['shape_gr'] = source_valleys_shp['LENGTH'] / source_valleys_shp['bas_area'] * 100.0
    
    #removal of redudant columns
    del source_valleys_shp['UP_ELEV']; del source_valleys_shp['DN_ELEV']; del source_valleys_shp['LENGTH']; del source_valleys_shp['blind']; del source_valleys_shp['shortLen']; del source_valleys_shp['slope_15']; del source_valleys_shp['sl15_count']; del source_valleys_shp['count']; del source_valleys_shp['bas_area']; del source_valleys_shp['azi']; 
    
    # save inter valley
    source_valleys_out =  wdir + 'source_valleys_params.shp'
    source_valleys_shp.to_file(source_valleys_out)  
    
    with out3:
        display("source_valleys_params.shp EXPORTED")
      
    
    #removal of redudant columns
    del bas_shp['gl_count']; del bas_shp['count0']; del bas_shp['slope35']; del bas_shp['slope>35']; del bas_shp['el2500']; del bas_shp['el_max']; del bas_shp['el_min']; 
    
    # save the basins result
    bas_out =  wdir + 'basin_params.shp'
    bas_shp.to_file(bas_out, driver = 'ESRI Shapefile')
    
    with out3:
        display("basin_params.shp EXPORTED")
    
    #removal of redudant columns
    del source_basin_shp['count0']; del source_basin_shp['slope35']; del source_basin_shp['slope>35']; del source_basin_shp['el2500']; del source_basin_shp['el_max']; del source_basin_shp['el_min']; del source_basin_shp['gl_count'];

    
    # save the basins result
    source_basin_out =  wdir + 'source_basin_params.shp'
    source_basin_shp.to_file(source_basin_out, driver = 'ESRI Shapefile')
    
    with out3:
        display("source_basin_params.shp EXPORTED")
    
    slope_ras.close()  
    dem_ras.close()
    plan_ras.close()
    prof_ras.close()
    
    # PARAMETERS ---- source valleys ----------------------------------------------------------------- 
    # ----------------------------------------------------------------------------------------
        
      
    #delete preliminary data
    if delete == True:
        os.chdir(wdir)       
        try:
            os.remove(dem2500path)
        except OSError as error:
            print(dem2500path + " can not be removed")
        try:
            os.remove(slope_path)
        except OSError as error:
            print(slope_path + " can not be removed")
        try:
            os.remove(slope35path)
        except OSError as error:
            print(slope35path + " can not be removed")
        try:
            os.remove(ground_var_path)
        except OSError as error:
            print(ground_var_path + " can not be removed")    
        try:
            os.remove(planCurv_path)
        except OSError as error:
            print(planCurv_path + " can not be removed")   
        try:
            os.remove(planCurv_path2)
        except OSError as error:
            print(planCurv_path2 + " can not be removed")   
        try:
            os.remove(profileCurv_path)
        except OSError as error:
            print(profileCurv_path + " can not be removed")   
        try:
            os.remove(profileCurv_path2)
        except OSError as error:
            print(profileCurv_path2 + " can not be removed")   
        
      
         
    with out3: 
        out3.clear_output()
        printmd('**PROCESSING DONE**')
        printmd('All results saved in: ' + wdir +".")
        
    button2.disabled = False

In [6]:
def preparation():
    
    global wbt
    global wdir
    global dem
    global glac
    global basins
    global source_basins
    global inter_valley
    global source_valleys
    global delete    
  
    wbt = whitebox.WhiteboxTools()
    
    #input directory
    indir = fdialog.selected
    indir = indir.replace("\\", "/")    
    
    #input dem
    dem = fdialog2.selected
    dem = dem.replace("\\", "/")  
    
    #input glaciers
    glac = fdialog4.selected
    glac = glac.replace("\\", "/")  
    
    #working directory
    wdir = fdialog3.selected
    wdir = wdir.replace("\\", "/")
    wbt.work_dir = os.path.dirname(wdir)
    
    basins = indir + "basins2.shp"
    source_basins = indir + "sourceBasins.shp"
    inter_valley = indir + "intersect_valley2.shp"
    source_valleys = indir + "sourceValleys.shp"

    #delete preliminary data
    delete = check1.value    
    
    with out2:
        out2.clear_output()       
        calculation()

In [7]:
def on_button1_clicked(b):    
    with out3:
        out3.clear_output()
        printmd('computing ..')
    
    preparation()

def but1():
    
    global button1
    button1 = widgets.Button(description="OK", disabled=True)
    display(button1)    
    button1.on_click(on_button1_clicked)    

In [8]:
def activatebut1():
    
    if (fdialog2.selected[-4:] == '.tif') and fdialog4.selected[-4:] == '.tif':
        button1.disabled = False
    else:
        button1.disabled = True

In [9]:
def preliminary():
    
    global check1
    
    check1 = widgets.Checkbox(
        value=True,
        description='Delete preliminary data',
        disabled=False,
        indent=False
    )
    display(check1)

In [10]:
def change_dir():
    
    fdialog2.default_path = fdialog.selected
    fdialog2.reset()
    
    fdialog3.default_path = fdialog.selected
    fdialog3.reset()
    
    fdialog4.default_path = fdialog.selected
    fdialog4.reset()

In [11]:
def wkdchooser():
    
    global fdialog
    
    fdialog = FileChooser(
        os.getcwd(),        
        title='<b>Input working directory</b> (select folder with results from tool 1)',
        show_hidden=False,
        select_default=True,
        use_dir_icons=True,
        show_only_dirs=True
    )   
    
    display(fdialog) 
    fdialog.register_callback(change_dir)   
    

In [12]:
def demchooser():
    
    global fdialog2
    
    fdialog2 = FileChooser(
        os.getcwd(),
        title='<b>Input preprocessed dem</b> (select *.tif file)',
        show_hidden=False,
        select_default=True,
        use_dir_icons=True,
        show_only_dirs=False
    )   
    
    display(fdialog2) 
    fdialog2.register_callback(activatebut1)

In [13]:
def glacierchooser():
    
    global fdialog4
    
    fdialog4 = FileChooser(
        os.getcwd(),
        title='<b>Input glacier layer</b> (select *.tiff file)',
        show_hidden=False,
        select_default=True,
        use_dir_icons=True,
        show_only_dirs=False
    )   
    
    display(fdialog4) 
    fdialog4.register_callback(activatebut1)    
    

In [14]:
def wkdchooser2():
    
    global fdialog3
    
    fdialog3= FileChooser(
        os.getcwd(),       
        title='<b>Output working directory</b> (select folder)',
        show_hidden=False,
        select_default=True,
        use_dir_icons=True,
        show_only_dirs=True
    )
    
    display(fdialog3)   
    

In [15]:
def inputsetting():
    
    out1 = widgets.Output(layout={'border': '2px solid black'})
    display(out1)   
    with out1:
        wkdchooser()
        demchooser() 
        glacierchooser()
        wkdchooser2()
        preliminary()
        but1()       

# FLOW2024 - 2 - calculate parameters

In [16]:
inputsetting()

Output(layout=Layout(border='2px solid black'))

In [17]:
processing()

Output(layout=Layout(border='2px solid black', height='300px'))

In [18]:
msg()

Output(layout=Layout(border='2px solid black', height='350px'))